# TP1 - SQL

***
**1. De la tabla [Person].[Contact], mostrar todos los datos de la tabla, ordenando por Apellido ascendente y nombre descendente.**

In [2]:
SELECT *
FROM Person.Contact
ORDER BY LastName, FirstName DESC;

(19972 rows affected)

Displaying Top 10 rows.

Total execution time: 00:00:01.241

ContactID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailAddress,EmailPromotion,Phone,PasswordHash,PasswordSalt,AdditionalContactInfo,rowguid,ModifiedDate
1012,0,Mr.,Syed,E,Abbas,NULL,syed0@adventure-works.com,0,926-555-0182,x62ugnRF3meCWKvtl0uZETETq94Ed1cy2IlkOzeclGk=,exVxghg=,NULL,ff284881-01c2-4c77-95a7-4db96f59bb70,2003-04-08 00:00:00.000
2,0,Ms.,Catherine,R.,Abel,NULL,catherine0@adventure-works.com,1,747-555-0171,zh3goJUbYsPv92k4bVZuJtlLHwuvpQtu6uNcjkKSdF8=,rpyd5Tw=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes"">These are additional phone and pager numbers for the customer.<act:telephoneNumber><act:number>206-555-2222</act:number><act:SpecialInstructions>On weekends, contact the manager at this number.</act:SpecialInstructions></act:telephoneNumber><act:telephoneNumber><act:number>206-555-1234</act:number></act:telephoneNumber><act:pager><act:number>206-555-1244</act:number><act:SpecialInstructions>Do not page between 9:00 a.m. and 5:00 p.m.</act:SpecialInstructions></act:pager>Customer provided this additional home address.<act:homePostalAddress><act:Street>P.O Box 5</act:Street><act:City>Edmonds</act:City><act:StateProvince>WA</act:StateProvince><act:PostalCode>98431</act:PostalCode><act:CountryRegion>USA</act:CountryRegion><act:SpecialInstructions>This is an alternative address for billing only.</act:SpecialInstructions></act:homePostalAddress><act:eMail><act:eMailAddress>Joe@sample.com</act:eMailAddress><act:SpecialInstructions>Do not send e-mail for urgent issues. Use telephone instead.</act:SpecialInstructions></act:eMail><crm:ContactRecord date=""2001-07-02Z"">Sales contacted this customer to explain new pricing.</crm:ContactRecord></AdditionalContactInfo>",d54e0552-c226-4c22-af3b-762ca854cdd3,2005-05-16 16:33:33.077
3,0,Ms.,Kim,NULL,Abercrombie,NULL,kim2@adventure-works.com,0,334-555-0137,4I8349R6c33cK+j1ef3dZt0JHOQ9MV7OvEXpCbwhQrQ=,rrgbG/U=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes"">Customer provided this telephone number for weekend access.<act:telephoneNumber><act:number>605-555-9877</act:number></act:telephoneNumber>secondary phone number.<act:pager><act:number>310-555-5432</act:number><act:SpecialInstructions>Page only if other contact attempts fail.</act:SpecialInstructions></act:pager>Additional home address.<act:homePostalAddress><act:Street>990 5th Avenue</act:Street><act:City>Redmond</act:City><act:StateProvince>WA</act:StateProvince><act:PostalCode>98052</act:PostalCode><act:CountryRegion>USA</act:CountryRegion><act:SpecialInstructions>Use this address for billing only.</act:SpecialInstructions></act:homePostalAddress><act:eMail><act:eMailAddress>Customer3@sample.com</act:eMailAddress><act:SpecialInstructions>Do not send e-mails for urgent issues. Use telephone instead.</act:SpecialInstructions></act:eMail><crm:ContactRecord date=""2002-01-01Z"">Customer first contacted the sales department. This customer is interested in opening a new bicycle store.</crm:ContactRecord><crm:ContactRecord date=""2002-01-05Z"">Linda in sales returned the call. Customer provided another telephone number for off-hour coverage.<act:telephoneNumber><act:number>302-555-7733</act:number></act:telephoneNumber>We are still negotiating possible long-term contract.</crm:ContactRecord></AdditionalContactInfo>",f7cbdb48-0b44-470e-9f37-7060446fbfb9,2005-05-16 16:33:33.077
1212,0,NULL,Kim,B,Abercrombie,NULL,kim1@adventure-works.com,2,208-555-0114,vjjIYGB99tLATbne8D7MRTg2+tkExYHjmH/pcWSg3nE=,wdPlbgo=,NULL,9a2163b3-2f4d-4f9a-91bd-07d326140f9c,200

***
**2. De la tabla [Person].[Contact], muestre los nombres de las personas que comienzan con 'D' o que comienzan con "A", ordenado alfabéticamente.**

In [3]:
SELECT pc.FirstName
FROM Person.Contact AS pc
WHERE pc.FirstName LIKE 'D%' OR pc.FirstName LIKE 'A%'
ORDER BY 1;

(3228 rows affected)

Displaying Top 10 rows.

Total execution time: 00:00:00.086

FirstName
A.
A.
A. Scott
Aaron
Aaron
Aaron
Aaron
Aaron
Aaron
Aaron


***
**3 - De la tabla [Person].[Contact],muestra todos los registros que se modificaron después del 1/1/2013.**

In [6]:
--Modifico año a 2003 porque las modificacines van entre 1997 y 2005
SELECT *
FROM Person.Contact 
WHERE ModifiedDate > '01-01-2003';

(15061 rows affected)

Displaying Top 10 rows.

Total execution time: 00:00:00.612

ContactID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailAddress,EmailPromotion,Phone,PasswordHash,PasswordSalt,AdditionalContactInfo,rowguid,ModifiedDate
1,0,Mr.,Gustavo,NULL,Achong,NULL,gustavo0@adventure-works.com,2,398-555-0132,GylyRwiKnyNPKbC1r4FSqA5YN9shIgsNik5ADyqStZc=,TVGHbhY=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes""><act:telephoneNumber><act:number>425-555-1112</act:number><act:SpecialInstructions>Call only after 5:00 p.m.</act:SpecialInstructions></act:telephoneNumber>Note that the customer has a secondary home address.<act:homePostalAddress><act:Street>123 Oak</act:Street><act:City>Seattle</act:City><act:StateProvince>WA</act:StateProvince><act:PostalCode>98001</act:PostalCode><act:CountryRegion>USA</act:CountryRegion><act:SpecialInstructions>If correspondence to the primary address fails, try this one.</act:SpecialInstructions></act:homePostalAddress>Customer provided additional email address.<act:eMail><act:eMailAddress>customer1@sample.com</act:eMailAddress><act:SpecialInstructions>For urgent issues, do not send e-mail. Instead use this emergency contact phone<act:telephoneNumber><act:number>425-555-1111</act:number></act:telephoneNumber>.</act:SpecialInstructions></act:eMail><crm:ContactRecord date=""2001-06-02Z"">This customer is interested in purchasing high-end bicycles for his family. The customer contacted Michael in sales.</crm:ContactRecord></AdditionalContactInfo>",d4c132d3-fcb5-4231-9dd5-888a54bec693,2005-05-16 16:33:33.060
2,0,Ms.,Catherine,R.,Abel,NULL,catherine0@adventure-works.com,1,747-555-0171,zh3goJUbYsPv92k4bVZuJtlLHwuvpQtu6uNcjkKSdF8=,rpyd5Tw=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes"">These are additional phone and pager numbers for the customer.<act:telephoneNumber><act:number>206-555-2222</act:number><act:SpecialInstructions>On weekends, contact the manager at this number.</act:SpecialInstructions></act:telephoneNumber><act:telephoneNumber><act:number>206-555-1234</act:number></act:telephoneNumber><act:pager><act:number>206-555-1244</act:number><act:SpecialInstructions>Do not page between 9:00 a.m. and 5:00 p.m.</act:SpecialInstructions></act:pager>Customer provided this additional home address.<act:homePostalAddress><act:Street>P.O Box 5</act:Street><act:City>Edmonds</act:City><act:StateProvince>WA</act:StateProvince><act:PostalCode>98431</act:PostalCode><act:CountryRegion>USA</act:CountryRegion><act:SpecialInstructions>This is an alternative address for billing only.</act:SpecialInstructions></act:homePostalAddress><act:eMail><act:eMailAddress>Joe@sample.com</act:eMailAddress><act:SpecialInstructions>Do not send e-mail for urgent issues. Use telephone instead.</act:SpecialInstructions></act:eMail><crm:ContactRecord date=""2001-07-02Z"">Sales contacted this customer to explain new pricing.</crm:ContactRecord></AdditionalContactInfo>",d54e0552-c226-4c22-af3b-762ca854cdd3,2005-05-16 16:33:33.077
3,0,Ms.,Kim,NULL,Abercrombie,NULL,kim2@adventure-works.com,0,334-555-0137,4I8349R6c33cK+j1ef3dZt0JHOQ9MV7OvEXpCbwhQrQ=,rrgbG/U=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes"">Customer provided this telephone number for weekend access.<act:telephoneNumber><act:number>605-555-9877</act:number></act:telephoneNumber>secondary phone number.<act:pager><act:number>310-555-

***
**4 - De la tabla [Person]. [Address], muestre los nombres de las ciudades sin repetirlos.**

In [7]:
SELECT DISTINCT City 
FROM Person.Address;

(575 rows affected)

Displaying Top 10 rows.

Total execution time: 00:00:00.054

City
Cheltenham
Kingsport
Suresnes
Baltimore
Reading
Fontana
Braunschweig
Sooke
Newark
Oxnard


***
**5 - Mostrar la cantidad de ventas que se realizaron con cada tipo de tarjetas de credito (se deben trabajar las tablas SalesOrderHeader y CreditCard).**

In [42]:
SELECT cc.CardType, COUNT(*) AS cantidad_ventas
FROM Sales.SalesOrderHeader soh
JOIN Sales.CreditCard cc
    ON  soh.CreditCardID = cc.CreditCardID
GROUP BY cc.CardType;

(4 rows affected)

Total execution time: 00:00:00.104

CardType,cantidad_ventas
SuperiorCard,7634
Vista,7432
Distinguish,7590
ColonialVoice,7678


In [40]:
-- Verifico que coincide con el total de ventas con tarjeta y que hay ventas sin tarjeta
SELECT COUNT(*) num_vtas_con_tarjeta
FROM  Sales.SalesOrderHeader
WHERE CreditCardID IS NOT NULL;

SELECT COUNT(*) num_vtas_sin_tarjeta
FROM  Sales.SalesOrderHeader
WHERE CreditCardID IS NULL;

SELECT COUNT(*) num_vtas_total
FROM  Sales.SalesOrderHeader;

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.076

num_vtas_con_tarjeta
30334


num_vtas_sin_tarjeta
1131


num_vtas_total
31465


***
**6 - De la tabla [Sales]. [CreditCard], muestre todas las tarjetas con un año de vencimiento anterior al 2018.**

In [22]:
-- Reemplazo 2018 por 2006 porque los vencimientos van entre 2005 y 2008
SELECT *
FROM Sales.CreditCard
WHERE ExpYear < '2006';

(4743 rows affected)

Displaying Top 10 rows.

Total execution time: 00:00:00.136

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate
2,Distinguish,55552127249722,8,2005,2004-01-06 00:00:00.000
3,ColonialVoice,77778344838353,7,2005,2004-02-15 00:00:00.000
5,Vista,11114404600042,4,2005,2003-03-05 00:00:00.000
9,Distinguish,55553465625901,2,2005,2003-10-25 00:00:00.000
13,Vista,11119775847802,11,2005,2004-04-01 00:00:00.000
22,ColonialVoice,77772030376004,3,2005,2004-01-25 00:00:00.000
24,Vista,11111658051128,11,2005,2003-09-22 00:00:00.000
39,Vista,11117546738833,5,2005,2003-10-15 00:00:00.000
42,Distinguish,55555053963314,3,2005,2003-08-02 00:00:00.000
46,SuperiorCard,33331306076760,12,2005,2004-01-08 00:00:00.000


***
**7 - Traer de la tabla Person.Contact los datos de contacto de todos los que NO son empleados (HumanResources.Employee)**

In [76]:
SELECT pc.*
FROM Person.Contact pc
LEFT JOIN HumanResources.Employee hre
    ON pc.ContactID = hre.ContactID
WHERE hre.ContactID IS NULL;

(19682 rows affected)

Displaying Top 10 rows.

Total execution time: 00:00:00.780

ContactID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailAddress,EmailPromotion,Phone,PasswordHash,PasswordSalt,AdditionalContactInfo,rowguid,ModifiedDate
1,0,Mr.,Gustavo,NULL,Achong,NULL,gustavo0@adventure-works.com,2,398-555-0132,GylyRwiKnyNPKbC1r4FSqA5YN9shIgsNik5ADyqStZc=,TVGHbhY=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes""><act:telephoneNumber><act:number>425-555-1112</act:number><act:SpecialInstructions>Call only after 5:00 p.m.</act:SpecialInstructions></act:telephoneNumber>Note that the customer has a secondary home address.<act:homePostalAddress><act:Street>123 Oak</act:Street><act:City>Seattle</act:City><act:StateProvince>WA</act:StateProvince><act:PostalCode>98001</act:PostalCode><act:CountryRegion>USA</act:CountryRegion><act:SpecialInstructions>If correspondence to the primary address fails, try this one.</act:SpecialInstructions></act:homePostalAddress>Customer provided additional email address.<act:eMail><act:eMailAddress>customer1@sample.com</act:eMailAddress><act:SpecialInstructions>For urgent issues, do not send e-mail. Instead use this emergency contact phone<act:telephoneNumber><act:number>425-555-1111</act:number></act:telephoneNumber>.</act:SpecialInstructions></act:eMail><crm:ContactRecord date=""2001-06-02Z"">This customer is interested in purchasing high-end bicycles for his family. The customer contacted Michael in sales.</crm:ContactRecord></AdditionalContactInfo>",d4c132d3-fcb5-4231-9dd5-888a54bec693,2005-05-16 16:33:33.060
2,0,Ms.,Catherine,R.,Abel,NULL,catherine0@adventure-works.com,1,747-555-0171,zh3goJUbYsPv92k4bVZuJtlLHwuvpQtu6uNcjkKSdF8=,rpyd5Tw=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes"">These are additional phone and pager numbers for the customer.<act:telephoneNumber><act:number>206-555-2222</act:number><act:SpecialInstructions>On weekends, contact the manager at this number.</act:SpecialInstructions></act:telephoneNumber><act:telephoneNumber><act:number>206-555-1234</act:number></act:telephoneNumber><act:pager><act:number>206-555-1244</act:number><act:SpecialInstructions>Do not page between 9:00 a.m. and 5:00 p.m.</act:SpecialInstructions></act:pager>Customer provided this additional home address.<act:homePostalAddress><act:Street>P.O Box 5</act:Street><act:City>Edmonds</act:City><act:StateProvince>WA</act:StateProvince><act:PostalCode>98431</act:PostalCode><act:CountryRegion>USA</act:CountryRegion><act:SpecialInstructions>This is an alternative address for billing only.</act:SpecialInstructions></act:homePostalAddress><act:eMail><act:eMailAddress>Joe@sample.com</act:eMailAddress><act:SpecialInstructions>Do not send e-mail for urgent issues. Use telephone instead.</act:SpecialInstructions></act:eMail><crm:ContactRecord date=""2001-07-02Z"">Sales contacted this customer to explain new pricing.</crm:ContactRecord></AdditionalContactInfo>",d54e0552-c226-4c22-af3b-762ca854cdd3,2005-05-16 16:33:33.077
3,0,Ms.,Kim,NULL,Abercrombie,NULL,kim2@adventure-works.com,0,334-555-0137,4I8349R6c33cK+j1ef3dZt0JHOQ9MV7OvEXpCbwhQrQ=,rrgbG/U=,"<AdditionalContactInfo xmlns=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactInfo"" xmlns:crm=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactRecord"" xmlns:act=""http://schemas.microsoft.com/sqlserver/2004/07/adventure-works/ContactTypes"">Customer provided this telephone number for weekend access.<act:telephoneNumber><act:number>605-555-9877</act:number></act:telephoneNumber>secondary phone number.<act:pager><act:number>310-555-

***
8 - Crear una tabla [Test].[Tarjetas_vencidas] y cargar en la misma el resultado de la consulta 7

In [56]:
-- Asumo que se refiere a la consulta 6
-- Evito notación punto ara no tener que crear una schema.
SELECT *
INTO Test_Tarjetas_vencidas 
FROM Sales.CreditCard   
WHERE ExpYear < '2006';

(4743 rows affected)

Total execution time: 00:00:00.040

In [46]:
--Verifico
SELECT TOP(5) *
FROM Test_Tarjetas_vencidas;

(5 rows affected)

Total execution time: 00:00:00.016

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate
2,Distinguish,55552127249722,8,2005,2004-01-06 00:00:00.000
3,ColonialVoice,77778344838353,7,2005,2004-02-15 00:00:00.000
5,Vista,11114404600042,4,2005,2003-03-05 00:00:00.000
9,Distinguish,55553465625901,2,2005,2003-10-25 00:00:00.000
13,Vista,11119775847802,11,2005,2004-04-01 00:00:00.000


***
9 - En la tabla [Test].[Tarjetas_vencidas] crear una nueva columna "bandera" con tipo int

In [57]:
-- Uso 0 como valor default
ALTER TABLE Test_Tarjetas_vencidas ADD bandera INT NOT NULL DEFAULT(0);

Commands completed successfully.

Total execution time: 00:00:00.040

In [58]:
--Verifico
SELECT TOP(5) *
FROM Test_Tarjetas_vencidas;

(5 rows affected)

Total execution time: 00:00:00.098

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate,bandera
10079,Distinguish,55551552070407,7,2005,2004-06-29 00:00:00.000,0
10082,ColonialVoice,77773275653296,7,2005,2003-09-04 00:00:00.000,0
10084,Distinguish,55553734775324,12,2005,2003-11-20 00:00:00.000,0
10090,ColonialVoice,77779811715774,9,2005,2004-04-03 00:00:00.000,0
10093,SuperiorCard,33335858015470,7,2005,2001-08-06 00:00:00.000,0


***
10 - Actualizar la tabla Test.Tarjetas_vencidas y dar valor 1 a la columna "bandera" cuando el vencimiento anterior del año 2015

In [59]:
--Elijo 2006 en lugar de 2015 porque los vencimientos van entre 2005 y 2008
UPDATE Test_Tarjetas_vencidas
SET bandera = 1
WHERE ExpYear < '2006';

(4743 rows affected)

Total execution time: 00:00:00.385

In [61]:
--Verifico
SELECT TOP(5) *
FROM Test_Tarjetas_vencidas
ORDER BY bandera DESC;

(5 rows affected)

Total execution time: 00:00:00.015

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate,bandera
10097,Vista,11118641082223,9,2005,2004-05-12 00:00:00.000,1
10093,SuperiorCard,33335858015470,7,2005,2001-08-06 00:00:00.000,1
10090,ColonialVoice,77779811715774,9,2005,2004-04-03 00:00:00.000,1
10084,Distinguish,55553734775324,12,2005,2003-11-20 00:00:00.000,1
10082,ColonialVoice,77773275653296,7,2005,2003-09-04 00:00:00.000,1


In [ ]:
--Para eliminarla
DROP TABLE Test_Tarjetas_vencidas;

***
**11 - crear una vista donde se muestre lo desarrollado en el punto 2.**

In [63]:
CREATE VIEW test_view
AS SELECT pc.FirstName
FROM person.Contact AS pc
WHERE pc.FirstName LIKE 'D%' or pc.FirstName LIKE 'A%';

Commands completed successfully.

Total execution time: 00:00:00.010

In [64]:
--Chequeo que está
SELECT TOP(5) *
FROM test_view;

(5 rows affected)

Total execution time: 00:00:00.022

FirstName
Amy
Anna
Daniel
Douglas
Alberto


In [65]:
--Para eliminarla
DROP VIEW IF EXISTS test_view;

Commands completed successfully.

Total execution time: 00:00:00.010

**12 - Crear una vista donde se muestren los datos de los empleados cuyo cumpleaños sea el dia de "hoy", Mostrando la fecha de nacimiento con el formato análogo a “15/11/2021”.**

In [96]:
CREATE VIEW birthday_view AS
SELECT FORMAT(hre.BirthDate, 'dd/MM/yyyy') AS BirthDay, pc.*
FROM HumanResources.Employee hre
JOIN Person.Contact pc
      ON pc.ContactID = hre.ContactID
WHERE (MONTH(hre.BirthDate) = MONTH(GETDATE()) AND DAY(hre.BirthDate) = DAY(GETDATE())) OR
      (FORMAT(hre.BirthDate, 'dd/MM') = '29/02' AND FORMAT(GETDATE(), 'dd/MM') = '01/03');

Commands completed successfully.

Total execution time: 00:00:00.011

In [97]:
--Chequeo que está
SELECT *
FROM birthday_view;

(2 rows affected)

Total execution time: 00:00:00.116

BirthDay,ContactID,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailAddress,EmailPromotion,Phone,PasswordHash,PasswordSalt,AdditionalContactInfo,rowguid,ModifiedDate
01/06/1974,1286,0,NULL,Kim,T,Ralls,NULL,kim0@adventure-works.com,2,309-555-0129,+VjdxFESErROmRQkb/z7zfADRHKgyWA1ZBmGI89+Gpo=,rJYW3u4=,NULL,7ab39816-ff5a-4f3d-a3dc-88d906ec054f,1999-01-20 00:00:00.000
01/06/1943,1093,0,NULL,Prasanna,E,Samarawickrama,NULL,prasanna0@adventure-works.com,1,129-555-0199,2u5E5B9OVe0NyB6SRbL4mcEXMpH28+5jvZzEc439Un4=,1EleEGU=,NULL,e8673e92-31f6-42f4-a65c-a595d3f899f6,2000-02-16 00:00:00.000


In [95]:
--P Borrar
DROP VIEW IF EXISTS birthday_view;

Commands completed successfully.

Total execution time: 00:00:00.010

***
**12bis - A partir de la tabla SalesOrderHeader, Crear un Stored Procedure donde se muestren el dinero recaudado entre 2 fechas que se ingresaran por parametro.**


***
**13 - Traer el nombre de las tablas pertenecientes al esquema Production.**

***
**14 - Traer por codigo la query utilizada para crear la vista correspondiente al punto 11.**

**15 - Traer por interfaz la query utilizada para crear la tabla Person.Contact.**

***
**16 - Traer por Codigo la query utilizada para crear la tabla Person.Contact.**